<a href="https://colab.research.google.com/github/haugekugga14/DEM-analysis/blob/main/%E6%9D%A1%E6%9D%A1%E5%A4%A7%E8%B7%AF%E9%80%9A%E7%BD%97%E9%A9%AC%EF%BC%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

我一直在做一个关于公共交通网络的项目，以及如何识别这种网络的热点和瓶颈

上传 json

In [ ]:
DIRPATH = "/content/"

In [ ]:
import geopandas as gpd # version: 0.9.0import matplotlib.pyplot as plt # version: 3.7.1
gdf = gpd.read_file('dataverse_files-2')
gdf = gdf.to_crs(4326)
print(len(gdf))
gdf.head(3)

In [ ]:
f, ax = plt.subplots(1,1,figsize=(15,10))
gdf.plot(column = 'CERTAINTY', ax=ax)

2.将道路网络转换为图形对象
上图显示的公路网是一堆线状多边形。不过，为了量化罗马等地的重要性，我打算进行一些图形计算。这意味着我需要将这些线串转换成图形。

In [ ]:
# create an edge table
edges = gdf.copy()
edges['u'] = [str(g.coords[0][0]) + '_' + str(g.coords[0][1]) for g in edges.geometry.to_list()]
edges['v'] = [str(g.coords[-1][0]) + '_' + str(g.coords[-1][1]) for g in edges.geometry.to_list()]
edges_copy = edges.copy()
edges['key'] = range(len(edges))
edges = edges.set_index(['u', 'v', 'key'])
edges.head(3)

In [ ]:
import pandas as pd  # version: 1.4.2
from shapely.geometry import Point  # version: 1.7.1

# create a node table
nodes = pd.DataFrame(edges_copy['u'].append(edges_copy['v']), columns=['osmid'])
nodes['geometry'] = [Point(float(n.split('_')[0]), float(n.split('_')[1])) for n in nodes.osmid.to_list()]
nodes['x'] = [float(n.split('_')[0]) for n in nodes.osmid.to_list()]
nodes['y'] = [float(n.split('_')[1]) for n in nodes.osmid.to_list()]
nodes = gpd.GeoDataFrame(nodes)
nodes = nodes.set_index('osmid')
nodes.head(3)

创建图表

In [ ]:
import osmnx as ox  # version: 1.0.1

# Now build the graph
graph_attrs = {'crs': 'epsg:4326', 'simplified': True}
G = ox.graph_from_gdfs(nodes, edges[['geometry']], graph_attrs)
print(type(G))
print(G.number_of_nodes()), print(G.number_of_edges())

在这里，我成功地将 GIS 数据文件转换成了一个有 5122 个节点和 7154 条边的网络对象。现在，我想看一看。我们也可以使用 NetworkX 将网络可视化。不过，我更喜欢使用开源软件 Gephi。它提供了更大的灵活性和更好的可视化微调选项。让我们将 G 转换成与 Gephi 兼容的文件并导出它--在这个版本中，我将使用无权、无向图。

In [ ]:
# Transform and export the graph
import networkx as nx  # version: 2.5

G_clean = nx.Graph()
for u, v, data in G.edges(data=True):
    G_clean.add_edge(u, v)

G_clean2 = nx.Graph()
G_clean2.add_edges_from(G_clean.edges(data=True))

nx.write_gexf(G_clean2, 'roman_empire_network.gexf')

此外，我还创建了一个名为 coordiantes.csv 的数据表，其中保存了道路网络中每个节点（交叉口）的坐标。

In [ ]:
nodes2 = nodes[nodes.index.isin(set(G.nodes))].drop(columns=['geometry'])
nodes2.index.name = 'Id'
nodes2.to_csv('coordinates.csv')

### Gephi
在这种可视化图示中，每个节点都对应一个交叉点，颜色编码所谓的网络群落（相互密集链接的子图），而节点的大小则根据其间性中心度来确定。Betweenness 是一种网络中心度量，用于量化节点的桥梁作用。因此，节点越大，中心性越强。

在视觉上，我们还可以看到地理位置是如何推动集群发展的，以及令人惊讶的是，意大利也是如何自成一体的，这可能是因为意大利国内的公路网络要密集得多。

4.网络中心

欣赏完视觉效果后，让我们回到图本身并对其进行量化。在这里，我将计算每个节点的总度，衡量它所拥有的连接数，以及每个节点的非规范化中心度，计算穿过每个节点的最短路径总数。

In [ ]:
node_degrees = dict(G_clean2.degree)
node_betweenness = dict(nx.betweenness_centrality(G_clean2, normalized=False))

现在，我有了每个交叉点的重要性得分。此外，在节点表中，我们还找到了它们的位置--现在该是讨论主要问题的时候了。为此，我要量化每个节点位于罗马行政边界内的重要性。为此，我需要罗马的行政边界，从 OSMnx 上很容易获得（注意：今天的罗马可能与过去的罗马不同，但大致上应该没问题）。

In [ ]:
admin = ox.geocode_to_gdf('Rome, Italy')
admin.plot()

此外，从视觉效果上看，罗马显然不是道路网络中的单一节点，而是附近有许多节点。因此，我们需要某种分选和空间索引方法，帮助我们将属于罗马的所有道路网节点和交叉路口归类。此外，我们还希望这种分组在整个帝国范围内具有可比性。这就是为什么我不只是将节点映射到罗马的行政区域，而是采用 Uber 的 H3 六边形分类法，创建六边形网格。然后，将每个节点映射到所包围的六边形中，并根据所包围网络节点的中心性得分计算该六边形的综合重要性。最后，我将讨论最中心的六边形如何与罗马重叠。

In [ ]:
import alphashape  # version: 1.1.0
from descartes import PolygonPatch

# take a random sample of the node points
sample = nodes.sample(1000)
sample.plot()

# create its concave hull
points = [(point.x, point.y) for point in sample.geometry]
alpha = 0.95 * alphashape.optimizealpha(points)
hull = alphashape.alphashape(points, alpha)
hull_pts = hull.exterior.coords.xy

fig, ax = plt.subplots()
ax.scatter(hull_pts[0], hull_pts[1], color='red')
ax.add_patch(PolygonPatch(hull, fill=False, color='green'))

让我们把帝国的多边形分割成六边形网格：

In [ ]:
import h3  # version: 3.7.3
from shapely.geometry import Polygon  # version: 1.7.1
import numpy as np  # version: 1.22.4

def split_admin_boundary_to_hexagons(polygon, resolution):
    coords = list(polygon.exterior.coords)
    admin_geojson = {"type": "Polygon", "coordinates": [coords]}
    hexagons = h3.polyfill(admin_geojson, resolution, geo_json_conformant=True)
    hexagon_geometries = {hex_id: Polygon(h3.h3_to_geo_boundary(hex_id, geo_json=True)) for hex_id in hexagons}
    return gpd.GeoDataFrame(hexagon_geometries.items(), columns=['hex_id', 'geometry'])

roman_empire = split_admin_boundary_to_hexagons(hull, 3)
roman_empire.plot()

现在，将路网节点映射成六边形，并在每个六边形上附加中心性得分。然后我将每个六边形内每个节点的连接数和穿过这些节点的最短路径数相加，从而汇总出每个节点的重要性：

In [ ]:
gdf_merged = gpd.sjoin(roman_empire, nodes[['geometry']])
gdf_merged['degree'] = gdf_merged.index_right.map(node_degrees)
gdf_merged['betweenness'] = gdf_merged.index_right.map(node_betweenness)
gdf_merged = gdf_merged.groupby(by='hex_id')[['degree', 'betweenness']].sum()
gdf_merged.head(3)

最后，将综合中心度得分与帝国六边形地图相结合：

In [ ]:
roman_empire = roman_empire.merge(gdf_merged, left_on='hex_id', right_index=True, how='outer')
roman_empire = roman_empire.fillna(0)

并将其可视化。在这张可视化图中，我还添加了空网格作为底图，然后根据网格内路网节点的总重要性为每个网格单元着色。这样，着色时就会用绿色突出最重要的单元格。此外，我还添加了白色的罗马多边形。首先，按程度着色：

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(15, 15))
gpd.GeoDataFrame([hull], columns=['geometry']).plot(
    ax=ax, color='grey', edgecolor='k', linewidth=3, alpha=0.1
)
roman_empire.plot(column='degree', cmap='RdYlGn', ax=ax)
gdf.plot(ax=ax, color='k', linewidth=0.5, alpha=0.5)
admin.plot(ax=ax, color='w', linewidth=3, edgecolor='w')
ax.axis('off')
plt.savefig('degree.png', dpi=200)

现在，"之间 "染上了色彩：

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(15, 15))
gpd.GeoDataFrame([hull], columns=['geometry']).plot(
    ax=ax, color='grey', edgecolor='k', linewidth=3, alpha=0.1
)
roman_empire.plot(column='betweenness', cmap='RdYlGn', ax=ax)
gdf.plot(ax=ax, color='k', linewidth=0.5, alpha=0.5)
admin.plot(ax=ax, color='w', linewidth=3, edgecolor='w')
ax.axis('off')
plt.savefig('betweenness.png', dpi=200, bbox_inches='tight')

最后，我们得出了一个令人欣慰的结论。如果根据累加度给六边形单元着色，罗马的面积将遥遥领先。如果我们根据六边形的间距给它们上色，情况也差不多--罗马又占了上风。此外，连接罗马和中东的高速公路也是一个关键的高间隔区段。条条大路通罗马！